In [58]:
import tensorflow as tf
import cifar10, cifar10_input
from tensorflow.python.ops.image_ops_impl import ResizeMethod  
tf.reset_default_graph()

In [59]:
sess = tf.InteractiveSession()

In [60]:
model_dir = "/data/1xiu/models/cifar"
model_name = "cifar_model"

batch_size = 128
IMAGE_SIZE = 24  
#create inputs's placeholder， 特征和lable，batch_size在之后定义网络结构时用到了，所以数据尺寸中的第一个值即样本条数需要被预先设定
image_holder = tf.placeholder(tf.float32,  [batch_size, 24, 24,3])
label_holder = tf.placeholder(tf.int32, [batch_size])
    

In [61]:
def variable_with_weight_loss(shape, stddev, wl):
    var =tf.Variable(tf.truncated_normal(shape, stddev=stddev))
    if wl is not None:
        weight_loss = tf.multiply(tf.nn.l2_loss(var), wl,name = 'weight_loss')
        tf.add_to_collection('losses', weight_loss)
    return var

In [62]:
#the first network
weight1 = variable_with_weight_loss(shape=[5,5,3,64], stddev=5e-2,  wl=0.0)
kernel1 = tf.nn.conv2d(image_holder, weight1, [1,1,1,1], padding='SAME')
bias1 = tf.Variable(tf.constant(0.0, shape=[64]))
conv1 = tf.nn.relu(tf.nn.bias_add(kernel1, bias1))
pool1 = tf.nn.max_pool(conv1, ksize=[1,3,3,1], strides=[1,2,2,1], padding='SAME')
norm1 = tf.nn.lrn(pool1, 4,bias=1.0, alpha=0.001/9.0, beta=0.75)

In [63]:
#the sercond network
weight2 = variable_with_weight_loss(shape=[5,5,64,64], stddev=5e-2, wl=0.00)
kernel2 = tf.nn.conv2d(norm1, weight2, [1,1,1,1], padding='SAME')
bias2 = tf.Variable(tf.constant(0.1, shape=[64]))
conv2 = tf.nn.relu(tf.nn.bias_add(kernel2, bias2))
norm2 = tf.nn.lrn(conv2, 4,bias=1.0, alpha=0.001/9.0, beta=0.75)
pool2 = tf.nn.max_pool(norm2, ksize=[1,3,3,1], strides=[1,2,2,1], padding='SAME')

#after the conv layer, full connect layer
reshape = tf.reshape(pool2, [batch_size, -1])
dim = reshape.get_shape()[1].value
weight3 = variable_with_weight_loss(shape=[dim, 384], stddev=0.04, wl=0.004)
bias3 = tf.Variable(tf.constant(0.1, shape=[384]))
local3 = tf.nn.relu(tf.matmul(reshape, weight3)+bias3)

#this is full connect layer, too
weight4 = variable_with_weight_loss(shape=[384, 192], stddev=0.04, wl = 0.004)
bias4 = tf.Variable(tf.constant(0.1, shape=[192]))
local4 = tf.nn.relu(tf.matmul(local3, weight4) + bias4)

#the last layer
weight5 = variable_with_weight_loss(shape=[192, 10], stddev=1/192.0, wl=0.0)
bias5 = tf.Variable(tf.constant(0.0, shape=[10]))
logis = tf.add(tf.matmul(local4, weight5), bias5)

In [64]:

saver = tf.train.Saver()

saver.restore(sess, os.path.join(model_dir, model_name))

INFO:tensorflow:Restoring parameters from /data/1xiu/models/cifar/cifar_model


In [65]:
import numpy as np
data_dir = '/tmp/cifar10_data/cifar-10-batches-bin/'
# 从文件以字符串方式获取10个类标签，使用制表格分割  
cifar10_class = np.loadtxt(data_dir + "batches.meta.txt", str, delimiter='\t')  

In [66]:
def img_read(filename):  
    if not tf.gfile.Exists(filename):  
        tf.logging.fatal('File does not exists %s', filename)  
    image_data = tf.image.convert_image_dtype(tf.image.decode_jpeg(tf.read_file(filename),  
                                                                   channels=3), dtype=tf.float32)  
    height = IMAGE_SIZE  
    width = IMAGE_SIZE  
    image = tf.image.resize_images(image_data, (height, width), method=ResizeMethod.BILINEAR)  
    image = tf.expand_dims(image, -1)  
    image = tf.reshape(image, (1, 24, 24, 3))  
    return image  

In [91]:
import matplotlib.pyplot as plt
img = img_read("dog2.jpg")
#import cv2
#cv2.imread("dog2.jpg")


In [97]:

top_k_pred = tf.nn.top_k(logis, k=3)
output = sess.run(top_k_pred)
probability = np.array(output[0]).flatten()  # 取出概率值，将其展成一维数组
index = np.array(output[1]).flatten()

for i in np.arange(index.size):
    print([index[i], cifar10_class[index[i]], probability[i]])


InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder' with dtype float and shape [128,24,24,3]
	 [[Node: Placeholder = Placeholder[dtype=DT_FLOAT, shape=[128,24,24,3], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op u'Placeholder', defined at:
  File "/data/1xiu/anaconda2/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/data/1xiu/anaconda2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/data/1xiu/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/data/1xiu/anaconda2/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/data/1xiu/anaconda2/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/data/1xiu/anaconda2/lib/python2.7/site-packages/zmq/eventloop/

In [96]:
print(img)
top_k_pred = tf.nn.top_k(img, k=3)  
output = sess.run(top_k_pred) 
probability = np.array(output[0]).flatten()  # 取出概率值，将其展成一维数组  
index = np.array(output[1]).flatten()  
for i in np.arange(index.size):  
    print([index[i], cifar10_class[index[i]], probability[i]])  

Tensor("Reshape_12:0", shape=(1, 24, 24, 3), dtype=float32)
[0, 'airplane', 0.3921569]
[2, 'bird', 0.37647063]
[1, 'automobile', 0.34117648]
[0, 'airplane', 0.32156864]
[2, 'bird', 0.31633988]
[1, 'automobile', 0.28627452]
[2, 'bird', 0.3267974]
[0, 'airplane', 0.3137255]
[1, 'automobile', 0.29934642]
[2, 'bird', 0.34509805]
[0, 'airplane', 0.32941177]
[1, 'automobile', 0.32549021]
[0, 'airplane', 0.33464053]
[1, 'automobile', 0.30980393]
[2, 'bird', 0.30980393]
[0, 'airplane', 0.40522879]
[1, 'automobile', 0.34117648]
[2, 'bird', 0.32026145]
[0, 'airplane', 0.41960788]
[1, 'automobile', 0.3019608]
[2, 'bird', 0.25882354]
[0, 'airplane', 0.55424845]
[1, 'automobile', 0.4117648]
[2, 'bird', 0.34509814]
[0, 'airplane', 0.70065367]
[1, 'automobile', 0.56862754]
[2, 'bird', 0.48888895]
[0, 'airplane', 0.76470596]
[1, 'automobile', 0.66666669]
[2, 'bird', 0.58823532]
[0, 'airplane', 0.71241826]
[1, 'automobile', 0.67189544]
[2, 'bird', 0.59738564]
[0, 'airplane', 0.66013074]
[1, 'automobile

In [69]:
def evaluate_images(images):  # 执行验证  
   # logits = cifar10.inference(images, batch_size=1)  
    logits = cifar10.inference(images)
    with tf.Session() as sess:  
        # 预测最大的三个分类  
        top_k_pred = tf.nn.top_k(logits, k=3)  
        output = sess.run(top_k_pred)  
        probability = np.array(output[0]).flatten()  # 取出概率值，将其展成一维数组  
        index = np.array(output[1]).flatten()  
        # 使用表格的方式显示  
        tabel = PrettyTable(["index", "class", "probability"])  
        tabel.align["index"] = "l"    
        tabel.padding_width = 1   
        for i in np.arange(index.size):  
            tabel.add_row([index[i], cifar10_class[index[i]], probability[i]])  
        print tabel  
  

FailedPreconditionError: Attempting to use uninitialized value softmax_linear/biases
	 [[Node: softmax_linear/biases/read = Identity[T=DT_FLOAT, _class=["loc:@softmax_linear/biases"], _device="/job:localhost/replica:0/task:0/cpu:0"](softmax_linear/biases)]]

Caused by op u'softmax_linear/biases/read', defined at:
  File "/data/1xiu/anaconda2/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/data/1xiu/anaconda2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/data/1xiu/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/data/1xiu/anaconda2/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/data/1xiu/anaconda2/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/data/1xiu/anaconda2/lib/python2.7/sit